In [ ]:
import pandas as pd
import xarray as xr
import holoviews as hv
import os
import ipywidgets as ipw
import panel as pn
import panel.widgets as pnw
from holoviews import opts
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

import matplotlib.pyplot as plt
# import numpy as np
# import os
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
# import textwrap
# from matplotlib.patches import Rectangle
# # from matplotlib.colors import Normalize
import cv2



### check sample photograph (done manually)

In [ ]:
from IPython.display import Image, display

def display_image_with_grid_and_scale(image_path, grid_size=10, scale_length_cm=2, scale_bar_position_cm=(0, 0), figsize_multiplier=0.015):
    # Load the image
    image = cv2.imread(image_path)

    # Convert image to RGB (OpenCV uses BGR by default)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Get image dimensions
    height, width, _ = image_rgb.shape

    # Create a grid overlay
    for x in range(0, width, grid_size):
        cv2.line(image_rgb, (x, 0), (x, height), color=(255, 0, 0), thickness=2)

    for y in range(0, height, grid_size):
        cv2.line(image_rgb, (0, y), (width, y), color=(255, 0, 0), thickness=2)

    # Calculate proportional figsize
    figsize = (width * figsize_multiplier, height * figsize_multiplier)

    # Display the image with grid and scale bar
    plt.figure(figsize=figsize)
    plt.imshow(image_rgb)
    # plt.axis('on')  # Turn off axis labels
    plt.xticks(fontsize=35)                                                     # Set font size for X and Y axis labels
    plt.yticks(fontsize=35)
    # plt.yticks(fontsize=40)

    plt.show()

# Example: Display image with a grid overlay and a scale bar at a specified position


image_path = r"C:\Users\ife12309\OneDrive - Institutt for Energiteknikk\Pictures\Post Mortem photos\Lot 710\Statistical samples\710A(10)-bottom-1_map.png"
# image_path = r"C:\Users\ife12309\OneDrive - Institutt for Energiteknikk\Pictures\Post Mortem photos\Lot 710\Statistical samples\710A(10)-top-1_map.png"
# image_path = r"C:\Users\ife12309\OneDrive - Institutt for Energiteknikk\Pictures\Post Mortem photos\Lot 710\Statistical samples\710A(10)-top-annotated-1_map.png"
display_image_with_grid_and_scale(image_path, 
                                  grid_size=205,                                # grid size is self explanatory
                                                                                #scale_length_cm=2,             # I couldn't get these to work
                                                                                #scale_bar_position_cm=(50, 50), 
)



### load X, Y, Z data

In [3]:
# Step 1: Load the Data

# top of electrode
# file_path = r'C:\scripting\Profilometer\Lot 710A(10)\710A(10)Top-1_Height  - Thresholded (14.58 µm ; 146.3 µm).txt'
# file_path = r'C:\scripting\Profilometer\Lot 710A(10)\710A(10)Top-2_Height  - Thresholded (-18.28 µm ; 102.7 µm).txt'
# file_path = r'C:\scripting\Profilometer\Lot 710A(10)\710A(10)Top-2_Height  - Thresholded (-12.99 µm ; 31.90 µm).txt'
# file_path = r'C:\scripting\Profilometer\Lot 710A(10)\710A(10)Top-3_Height_Height  - Thresholded (-15.48 µm ; 88.99 µm).txt'

## bottom of electrode
# file_path = r'C:\scripting\Profilometer\Lot 710A(10)\710A(10)bottom-1_Height  - Thresholded (42.82 µm ; 162.3 µm).txt'
# file_path = r'C:\scripting\Profilometer\Lot 710A(10)\710A(10bottom-2_Height_Height  - Thresholded (-11.67 µm ; 120.0 µm).txt'
# file_path = r'C:\scripting\Profilometer\Lot 710A(10)\710A(10)bottom-3_Height  - Thresholded (-11.29 µm ; 40.30 µm).txt'
file_path = r'C:\scripting\Profilometer\Lot 710A(10)\710A(10)bottom-1_Height  - Thresholded (302.8 µm ; 474.9 µm).txt'



# df = pd.read_csv(file_path, sep='\t', names=['x', 'y', 'z'])  # original script, works with most files
df = pd.read_csv(file_path, sep='\t', names=['x', 'y', 'z'], dtype={'x': float, 'y': float, 'z': 'str'})
df['z'] = pd.to_numeric(df['z'], errors='coerce')
df = df.dropna()                                    # Option 1 - Drop rows with NaN values
# df['z'] = df['z'].fillna(0)                       # Option 2: Replace NaN values with 0 (use if you want to treat missing data as zero)

# Extract the file name from the path and take the first 50 characters
file_name = os.path.basename(file_path)
title = file_name[:50]

# Convert the dataframe to an Xarray Dataset
ds = xr.Dataset.from_dataframe(df.dropna().set_index(['x', 'y']))

In [ ]:
# Set the desired z-value limits (color limits)
z_min = -30  # Adjust this value as needed
z_max = 120   # Adjust this value as needed

# Create the heatmap with specified z-value limits and colorbar label
heatmap = hv.HeatMap(df, kdims=['x', 'y'], vdims='z')
heatmap.opts(cmap='seismic', colorbar=True, title=title, xlabel='X (mm)', ylabel='Y (mm)', 
             width=800, height=600, clim=(z_min, z_max),
             colorbar_opts={'title': 'Height (μm)'})  # Add units to the colorbar label

# Display the heatmap
heatmap



In [6]:
hv.save(heatmap, title+'.png', fmt='png')


In [ ]:
def plot_height_by_x(x):
    y_values = ds.sel(x=x)['y'].values
    z_values = ds.sel(x=x)['z'].values
    curve = hv.Curve((y_values, z_values), 'Y (mm)', 'Height (um)')
    return curve.opts(title=f'{title}__Height Profile at X = {x}', width=800, height=300)

def plot_height_by_y(y):
    x_values = ds.sel(y=y)['x'].values
    z_values = ds.sel(y=y)['z'].values
    curve = hv.Curve((x_values, z_values), 'X (mm)', 'Height (um)')
    return curve.opts(title=f'{title}__Height Profile at Y = {y}', width=800, height=300)

x_slider = pn.widgets.FloatSlider(start=float(ds['x'].min()), end=float(ds['x'].max()), step=0.1, name='X (mm)')
y_slider = pn.widgets.FloatSlider(start=float(ds['y'].min()), end=float(ds['y'].max()), step=0.1, name='Y (mm)')

x_stream = hv.streams.Stream.define('X', x=x_slider.value)()
y_stream = hv.streams.Stream.define('Y', y=y_slider.value)()

x_dmap = hv.DynamicMap(plot_height_by_x, streams=[x_stream])
y_dmap = hv.DynamicMap(plot_height_by_y, streams=[y_stream])

x_slider.param.watch(lambda event: x_stream.event(x=event.new), 'value')
y_slider.param.watch(lambda event: y_stream.event(y=event.new), 'value')

layout = pn.Column(
    heatmap,
    pn.Row(x_slider, y_slider),
    x_dmap,
    y_dmap
)
layout.servable()

# for later

import os

heatmap_savefile_path = r'C:\path\to\save\directory'

def save_heatmap(heatmap, filename, fmt='png'):
    # Combine the default path with the filename
    full_path = os.path.join(heatmap_savefile_path, filename)
    # Save the heatmap to the specified path
    hv.save(heatmap, full_path, fmt=fmt)
    print(f"Heatmap saved to: {full_path}")

save_heatmap(heatmap, 'my_heatmap.png')



In [6]:
# def plot_height_by_x(x):
#     y_values = ds.sel(x=x)['y'].values
#     z_values = ds.sel(x=x)['z'].values
#     curve = hv.Curve((y_values, z_values), 'Y (mm)', 'Height (um)')
#     return curve.opts(title=f'{title}__Height Profile at X = {x}', width=800, height=300)

# def plot_height_by_y(y):
#     x_values = ds.sel(y=y)['x'].values
#     z_values = ds.sel(y=y)['z'].values
#     curve = hv.Curve((x_values, z_values), 'X (mm)', 'Height (um)')
#     return curve.opts(title=f'{title}__Height Profile at Y = {y}', width=800, height=300)

# x_slider = pn.widgets.FloatSlider(start=float(ds['x'].min()), end=float(ds['x'].max()), step=0.1, name='X (mm)')
# y_slider = pn.widgets.FloatSlider(start=float(ds['y'].min()), end=float(ds['y'].max()), step=0.1, name='Y (mm)')

# x_stream = hv.streams.Stream.define('X', x=x_slider.value)()
# y_stream = hv.streams.Stream.define('Y', y=y_slider.value)()

# x_dmap = hv.DynamicMap(plot_height_by_x, streams=[x_stream])
# y_dmap = hv.DynamicMap(plot_height_by_y, streams=[y_stream])

# x_slider.param.watch(lambda event: x_stream.event(x=event.new), 'value')
# y_slider.param.watch(lambda event: y_stream.event(y=event.new), 'value')

# layout = pn.Column(
#     heatmap,
#     pn.Row(x_slider, y_slider),
#     x_dmap,
#     y_dmap
# )
# layout.servable()
